In [2]:
import sagemaker
import boto3
from sagemaker import get_execution_role

In [3]:
region = boto3.Session().region_name
sm_rt = boto3.Session().client('runtime.sagemaker', region_name=region)

ep_name = 'ct-model'
id_name = ''
target = 'reference'

file_name = "ct_test.csv"
test_file_name = "ct_test-no-label.csv"

kaggle_file_name = ""
kaggle_test_file_name = ""
kaggle_pred_file_name = ""

In [4]:
import pandas as pd
import numpy as np

df = pd.read_csv(file_name)
df = df.drop([target],axis=1) 
print(df.head())
print(df.shape)

df.to_csv(test_file_name, index=False)

   value0  value1    value2    value3    value4    value5    value6    value7  \
0     0.0     0.0  0.021710  0.959824  0.543624  0.837550  0.854570  0.946155   
1     0.0     0.0  0.062581  0.968990  0.611051  0.789786  0.831758  0.946945   
2     0.0     0.0  0.194260  0.000000  0.000000  0.000000  0.911728  0.970606   
3     0.0     0.0  0.289157  0.972871  0.575439  0.721380  0.831660  0.939199   
4     0.0     0.0  0.372760  0.967019  0.569170  0.686072  0.783167  0.923877   

     value8  value9  ...  value374  value375  value376  value377  value378  \
0  0.767379    0.00  ...       0.0       0.0       0.0       0.0       0.0   
1  0.793224    0.00  ...       0.0       0.0       0.0       0.0       0.0   
2  0.679942   -0.25  ...       0.0       0.0       0.0       0.0       0.0   
3  0.749918    0.00  ...       0.0       0.0       0.0       0.0       0.0   
4  0.769347    0.00  ...       0.0       0.0       0.0       0.0       0.0   

   value379  value380  value381  value382  v

In [5]:
f = open(test_file_name)
count = 0
pred = []

for line in f:
    if count==0:
        count=1
        continue
    response = sm_rt.invoke_endpoint(EndpointName=ep_name, ContentType='text/csv', Accept='text/csv', Body=line.encode('utf-8'))
    response = response['Body'].read().decode("utf-8")
    pred.append(float(response))
    count = count + 1


In [6]:
print(pred[:20])

import pandas as pd
import numpy as np
pd = pd.read_csv(file_name)
y_true = np.array(pd[target])
y_pred = np.array(pred)

from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_true, y_pred)
print("test mse = {}".format(mse))

[27.009599685668945, 27.253761291503906, 28.256681442260742, 27.423978805541992, 27.397663116455078, 28.293811798095703, 27.415180206298828, 27.29427719116211, 27.296205520629883, 28.02491569519043, 27.451629638671875, 27.374671936035156, 27.296947479248047, 27.15730857849121, 27.839614868164062, 27.530609130859375, 27.84438133239746, 27.63195037841797, 27.91532325744629, 27.833372116088867]
test mse = 18.992568621111197


In [6]:
import pandas as pd
import numpy as np

df = pd.read_csv(kaggle_file_name)

kaggle_test_ids = np.array(df[id_name])

df = df.drop([id_name],axis=1) 
print(df.head())
print(df.shape)

df.to_csv(kaggle_test_file_name, index=False)

FileNotFoundError: [Errno 2] File  does not exist: ''

In [14]:
f = open(kaggle_test_file_name)
count = 0
kaggle_pred = []

for line in f:
    if count == 0:
        count = 1
        continue
    response = sm_rt.invoke_endpoint(EndpointName=ep_name, ContentType='text/csv', Accept='text/csv', Body=line.encode('utf-8'))
    response = response['Body'].read().decode("utf-8")
    test_id = kaggle_test_ids[count-1]
    pred = int(response[0])
    kaggle_pred.append([test_id, pred])
    count = count + 1

kaggle_pred_df = pd.DataFrame(kaggle_pred, columns = [id_name, target])

kaggle_pred_df.to_csv(kaggle_pred_file_name, index=False)

In [5]:
print(kaggle_pred_df.head())
print(kaggle_pred_df.shape)

NameError: name 'kaggle_pred_df' is not defined

In [22]:
import boto3
sm = boto3.client('sagemaker')
#sm.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm.delete_endpoint(EndpointName=ep_name)

{'ResponseMetadata': {'RequestId': '3adbc464-12f4-4c5b-af4a-1bad8f737f35',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '3adbc464-12f4-4c5b-af4a-1bad8f737f35',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Wed, 24 Mar 2021 07:51:25 GMT'},
  'RetryAttempts': 0}}